# Nuclio - Process user location signal

## Setup the environment

In [2]:
# nuclio: ignore
import nuclio

### Define environment variables

In [1]:
import os

In [4]:
# nuclio: ignore
os.environ['STORES_TABLE'] = os.path.join(os.getenv('V3IO_USERNAME', 'iguazio'), 'stores')
os.environ['CUSTOMERS_TABLE'] = os.path.join(os.getenv('V3IO_USERNAME', 'iguazio'), 'customers')
os.environ['CUSTOMERS_STREAM'] = os.path.join(os.getenv('V3IO_USERNAME', 'iguazio'), 'customers_stream')
os.environ['PREDICTIONS_STREAM'] = os.path.join(os.getenv('V3IO_USERNAME', 'iguazio'), 'predictions')

In [5]:
# DB Acess
%nuclio env V3IO_API=${V3IO_FRAMESD}
%nuclio env V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Customers
%nuclio env COSTUMERS_STREAM=${CUSTOMERS_STREAM}
%nuclio env COSTUMERS_TABLE=${CUSTOMERS_TABLE}
%nuclio env STORES_TABLE=${STORES_TABLE}

# Predictions
%nuclio env PREDICTIONS_STREAM=${PREDICTIONS_STREAM}
%nuclio env PREDICTION_SERVER=http://prediction-server:8080

%nuclio: setting 'V3IO_API' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'COSTUMERS_STREAM' environment variable
%nuclio: setting 'COSTUMERS_TABLE' environment variable
%nuclio: setting 'STORES_TABLE' environment variable
%nuclio: setting 'PREDICTIONS_STREAM' environment variable
%nuclio: setting 'PREDICTION_SERVER' environment variable


### Base image

In [6]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Set cron trigger to read from stream

In [7]:
%nuclio config spec.triggers.secs.kind = "cron"
%nuclio config spec.triggers.secs.attributes.interval = "1m"

%nuclio: setting spec.triggers.secs.kind to 'cron'
%nuclio: setting spec.triggers.secs.attributes.interval to '1m'


### Install packages

In [8]:
%%nuclio cmd -c
pip install v3io_frames==0.6.13
pip install v3io==0.1.1 --upgrade
pip install requests==2.23.0
pip install pandas==1.0.1

## Function code

In [9]:
import json
import os
import requests
import time

# Data handling
import pandas as pd

# DB
import v3io
import v3io.dataplane
import v3io.logger
import v3io_frames as v3f

### Init context

In [10]:
def init_context(context):
    # DB Contexts
   
    v3c = v3io.dataplane.Context(v3io.logger.Logger('DEBUG')).new_session().new_container('users')
    setattr(context, 'v3c', v3c)
   
    v3c_frames = v3f.Client('framesd:8081', container='users')
    setattr(context, 'v3f', v3c_frames)
    
    
    # DB Tables
    
    customers_table = os.environ['COSTUMERS_TABLE']
    setattr(context, 'customers', customers_table)
    
    stores_table = os.environ['STORES_TABLE']
    setattr(context, 'stores', stores_table)
    
    predictions_stream = os.environ['PREDICTIONS_STREAM']
    setattr(context, 'predictions', predictions_stream)
    
    customers_stream = os.environ['COSTUMERS_STREAM']
    setattr(context, 'customers_stream', customers_stream)
    
    
    # Prediction server
    
    prediction_server = os.getenv('PREDICTION_SERVER')
    setattr(context, 'prediction_server', prediction_server)

### Helper functions

In [11]:
def is_customer_in_store(customer, context) -> bool:
    store_location = customer['location']
    store = context.v3f.read('kv', context.stores, filter=f'__name=="{store_location}"')

    return not store.empty

In [12]:
def is_customer_out_of_store(context, new_customer_locations):
    if not new_customer_locations.empty:
        users = new_customer_locations['id'].values.astype('int').astype('str')
        filter_line = str(list(users))
        filter_line = f'__name IN ({filter_line[1:-1]})'
        old_customer_locations = context.v3f.read('kv', context.customers, columns=['location'], filter=filter_line)
        old_customer_locations['is_store'] = old_customer_locations.apply(is_customer_in_store, args=[context], axis=1)
        return old_customer_locations[old_customer_locations['is_store'] == True]['location']

In [13]:
def update_customer_location(context, customer_id: str, location: str):
    context.v3f.execute('kv', context.customers , 'update', args={'key':customer_id, 'expression': f'SET location="{location}"', 'condition':''})

In [14]:
def update_store_count(customer, context, is_add=True):
    operator = '+' if is_add else '-'
    context.v3f.execute('kv', context.stores , 'update', args={'key': customer, 'expression': f'SET customers=customers{operator}1', 'condition':''})

In [15]:
def save_predictions(context, customer_id: str, prediction: pd.DataFrame):
    context.v3f.write('tsdb', context.predictions, prediction)

### Handler

In [26]:
def handler(context, event):
   
    # Get latest customer locations from the customers stream
    customers_stream = context.v3f.read('tsdb', context.customers_stream, start='now-1m')
    
    if not customers_stream.empty:
        # Has anyone moved out of any store?
        stores_to_update = is_customer_out_of_store(context, customers_stream)
        stores_to_update.apply(update_store_count, args=[context, False])

        # Update the customer's new location
        [update_customer_location(context, str(int(customer['id'])), customer['location']) for idx, customer in customers_stream.iterrows()]
       
    
        # Get all customers that are in stores
        customers_stream['is_store'] = customers_stream.apply(is_customer_in_store, args=[context], axis=1)
        customers_stream = customers_stream[customers_stream['is_store']]
       
        # Update customers in stores count
        customers_stream['update_stores'] = customers_stream['location'].apply(update_store_count, args=[context])
       
        context.logger.debug(customers_stream)
 
        [requests.post(context.prediction_server, json={'id': str(int(customer['id'])), 'store': str(customer['location'])}) for idx, customer in customers_stream.iterrows()]

In [17]:
# nuclio: ignore
init_context(context)

In [27]:
# nuclio: ignore
event = nuclio.Event()
handler(context, event)

In [19]:
%nuclio deploy -n process_user_location -p recommendation_engine

[nuclio.deploy] 2019-08-11 13:38:34,372 (info) Building processor image
[nuclio.deploy] 2019-08-11 13:39:04,748 (info) Pushing image
[nuclio.deploy] 2019-08-11 13:39:18,908 (info) Build complete
[nuclio.deploy] 2019-08-11 13:39:22,100 done creating process-user-location, function address: 3.120.15.118:32537
%nuclio: function deployed
